<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD;
    color: black;"> <img style="float: left; padding-right: 10px;" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png" height="50px"> <a href='https://harvard-iacs.github.io/2023-AC215/' target='_blank'><strong><font color="#A41034">AC215: Productionizing AI (MLOps)</font></strong></a></h1>

# **<font color="#A41034">Tutorial - Data Pipelines, Labeling, Versioning, Cloud Storage</font>**

**Harvard University**<br/>
**Fall 2023**<br/>
**Instructor:**<br/>
Pavlos Protopapas

<hr style="height:2pt">

## **<font color="#A41034">Tutorial Outline</font>**

## **Learning Objectives**

By the end of this notebook, you will be:
* Familiar with data version from a DVC repo


## **Tutorial Content**

In order to run this tutorial you should have completed [Data Labeling & Versioning](https://github.com/dlops-io/data-labeling) steps.

- View data from a remote DVC repo
- Retrieve data from a remote DVC repo
- Retrieve a specifi data version from DVC


## **<font color="#A41034">Setup Notebook</font>**

**Copy & setup Colab**

1) Select "File" menu and pick "Save a copy in Drive"   

**Installs**

In [ ]:
!pip install dvc dvc-gs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.8/425.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) .

**Imports**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Colab auth
from google.colab import auth
from google.cloud import storage

**Utils**

Here are some util functions that we will be using in this notebook

In [ ]:
def compute_dataset_metrics(dataset_path):

  label_names = os.listdir(dataset_path)
  print("Labels:", label_names)

  # Generate a list of labels and path to images
  data_list = []
  for label in label_names:
    # Images
    image_files = os.listdir(os.path.join(dataset_path,label))
    data_list.extend([(label,os.path.join(dataset_path,label,f)) for f in image_files])

  data_list_with_metrics = []
  for item in data_list:
    # Read image
    image = cv2.imread(item[1])
    data_list_with_metrics.append((item[0],item[1],image.nbytes / (1024 * 1024.0)))

  # Build a dataframe
  data_list_with_metrics = np.asarray(data_list_with_metrics)
  dataset_df = pd.DataFrame({
    'label': data_list_with_metrics[:, 0],
    'path': data_list_with_metrics[:, 1],
    'size': data_list_with_metrics[:, 2],
    })

  dataset_df["size"] = dataset_df["size"].astype(float)

  dataset_mem_size = dataset_df["size"].sum()
  value_counts = dataset_df["label"].value_counts()

  print("Dataset Metrics:")
  print("----------------")
  print("Label Counts:")
  print(value_counts)
  print("Size in memory:",round(dataset_df["size"].sum(),2),"MB")

## **<font color="#A41034">Authenticate</font>**

Here we authenticate your Google account that has access to your GCS bucket

In [ ]:
# This step is required for DVC in colab to access your Bucket
auth.authenticate_user()

## **View Remote Data**

To view a DVC repository, listing data files and directories tracked by DVC alongside the remaining Git repo contents.

[dvc list reference](https://dvc.org/doc/command-reference/list)

In [ ]:
# Replace github url with your url
!dvc list -R https://github.com/dlops-io/data-versioning

.dvcignore
.gitignore
Dockerfile
Pipfile
Pipfile.lock
README.md
cli.py
docker-shell.bat
docker-shell.sh
mushroom_dataset.dvc
mushroom_dataset/amanita/1.jpg
mushroom_dataset/crimini/10.jpg
mushroom_dataset/crimini/11.jpg


## **Retrieving Data**

Once DVC-tracked data are stored remotely, they can be downloaded with dvc get when needed.

[dvc get reference](https://dvc.org/doc/command-reference/get)

In [ ]:
!rm -rf mushroom_dataset
!dvc get https://github.com/dlops-io/data-versioning mushroom_dataset --force --rev dataset_v1

!
  0%|          |mushroom_dataset/amanita/1.jpg     0.00/? [00:00<?,        ?B/s]
mushroom_dataset/amanita/1.jpg:   0% 0.00/441k [00:00<?, ?B/s{'info': ''}]      
                                                                          
!
  0%|          |mushroom_dataset/crimini/10.jpg    0.00/? [00:00<?,        ?B/s]
mushroom_dataset/crimini/10.jpg:   0% 0.00/26.7k [00:00<?, ?B/s{'info': ''}]    
                                                                            
!
  0%|          |mushroom_dataset/crimini/11.jpg    0.00/? [00:00<?,        ?B/s]


In [ ]:
# Check the dataset
compute_dataset_metrics("mushroom_dataset")

Labels: ['amanita', 'crimini']
Dataset Metrics:
----------------
Label Counts:
crimini    2
amanita    1
Name: label, dtype: int64
Size in memory: 9.56 MB


## **Retrieve a different data version**

In [ ]:
!rm -rf mushroom_dataset
!dvc get https://github.com/dlops-io/data-versioning mushroom_dataset --force --quiet --rev dataset_v2

In [ ]:
# Check the dataset
compute_dataset_metrics("mushroom_dataset")

Labels: ['amanita', 'crimini', 'oyster']
Dataset Metrics:
----------------
Label Counts:
crimini    2
amanita    1
oyster     1
Name: label, dtype: int64
Size in memory: 11.7 MB
